In [1]:
import autogen
import os
from autogen import AssistantAgent, UserProxyAgent
from dotenv import load_dotenv

In [12]:
load_dotenv("azure.env")

config_list = [
    {
        "model": os.getenv("OPENAI_DEPLOYMENT_NAME"),  
        "base_url": os.getenv("OPENAI_API_BASE"), 
        "api_type": "azure", 
        "api_version": os.getenv("OPENAI_API_VERSION"), 
        "api_key": os.getenv("OPENAI_API_KEY")
 }
]

gpt4_config = {
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120,
}

: 

In [4]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the planner to discuss the plan. Plan execution needs to be approved by this admin.",
    code_execution_config=False,
)


In [ ]:
planner = autogen.AssistantAgent(
    name="Planner",
    system_message="""Planner. Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
                    The plan may involve an engineer who can write code and a scientist who doesn't write code.
                    Explain the plan first. Be clear which step is performed by an engineer, and which step is performed by a scientist.
                    """,
    llm_config=gpt4_config,
)

In [5]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=gpt4_config,
    system_message="""Engineer. You follow an approved plan. You write python/shell code to solve tasks. 
                    Wrap the code in a code block that specifies the script type. The user can't modify your code. 
                    So do not suggest incomplete code which requires others to modify. 
                    Don't use a code block if it's not intended to be executed by the executor.
                    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. 
                    Check the execution result returned by the executor.
                    If the result indicates there is an error, fix the error and output the code again. 
                    Suggest the full code instead of partial code or code changes. 
                    If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
                    Analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
                """,
)


In [6]:
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=gpt4_config,
    system_message="""Scientist. You follow an approved plan. You are able to categorize papers after seeing their abstracts printed. 
                    You don't write code.
                    """,
)


In [8]:
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "paper",
        "use_docker": False,
    },  
)

In [9]:

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. 
                    Check whether the plan includes adding verifiable info such as source URL.
                    """,
    llm_config=gpt4_config,
)


In [10]:

groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, scientist, planner, executor, critic], messages=[], max_round=50
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

user_proxy.initiate_chat(
    manager,
    message="""
find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.
""",
)

Admin (to chat_manager):


find papers on LLM applications from arxiv in the last week, create a markdown table of different domains.


--------------------------------------------------------------------------------
Planner (to chat_manager):

**Plan Explanation:**

The plan involves two main roles: an engineer and a scientist. The engineer will be responsible for writing code to automate the process of finding and categorizing papers from arXiv, while the scientist will provide domain expertise to ensure the accuracy of the categorization and to refine the search criteria if necessary.

**Initial Plan:**

1. **Engineer**: Write a script to query the arXiv API for papers related to Large Language Models (LLMs) that have been published in the last week. The script will filter results based on specific keywords and metadata such as submission date.

2. **Engineer**: Extract relevant information from each paper, such as the title, authors, arXiv identifier, and abstract.

3. **Scientist*

ChatResult(chat_id=None, chat_history=[{'content': '\nfind papers on LLM applications from arxiv in the last week, create a markdown table of different domains.\n', 'role': 'assistant'}, {'content': "**Plan Explanation:**\n\nThe plan involves two main roles: an engineer and a scientist. The engineer will be responsible for writing code to automate the process of finding and categorizing papers from arXiv, while the scientist will provide domain expertise to ensure the accuracy of the categorization and to refine the search criteria if necessary.\n\n**Initial Plan:**\n\n1. **Engineer**: Write a script to query the arXiv API for papers related to Large Language Models (LLMs) that have been published in the last week. The script will filter results based on specific keywords and metadata such as submission date.\n\n2. **Engineer**: Extract relevant information from each paper, such as the title, authors, arXiv identifier, and abstract.\n\n3. **Scientist**: Define a list of domains relevan